In [ ]:
import time
from pathlib import Path
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np

# conditional random fields
import ace_sklearn_crfsuite
from ace_sklearn_crfsuite import metrics

# self-made
import src.analysis as analysis
import src.anomaly_model as anomaly_model
import src.utils as utils
import src.sensor_model as sensor_model

working_path = Path().resolve()
layout_database_path = working_path / "layout_data"

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

def fall_feature_sliding(mat, time_step, window_len, nrt_type = "instantaneous"):
    """
    Calculate features for fall detection.
    The feature includes;
    1: non response time of all sensors at the present time
    2: maximum height of the edge of the non response time sequence in the left / right part of the window.

    Parameters
    ----------
    mat : nummpy.ndarray of bool
        Raw sensor data matrix.
        mat[i][j] = j-th sensor state at i-th time.
    time_step : float
        Time step length [seconds] of mat = mat[1][0] - mat[0][0].
    window_len : int
        Length of window.
        window_len is the number of columns in mat.
        window_len must be an odd number.
    mode : int
        "instantaneous" : instantaneous non response time
        "sum" : sum in the window
        "max" : max in the window
    
    Returns
    -------
    feature : numpy.ndarray
        (number of time, dimension of feature).
        feature[t] = feature vector of the t-th time.
        feature.shape[0] == mat.shape[0].
        Let w be a (window_len - 1)/2.
        feature.shape[i] is a vector made from mat[i-w/2:i+w/2+1] for any w/2 <= i <mat.shape[0]-w/2
        feature.shape[i] is a vector made from mat[0:i+w/2+1] for any i < w/2.
        feature.shape[i] is a vector made from mat[i-w/2:mat.shape[0]] for any i >= mat.shape[0]-w/2.
        sensor_num_dic saves the realtion between index of sensors and index of features.
    """
    feature = np.zeros((mat.shape[0], mat.shape[1]), dtype = np.int32)
    for i, item in enumerate(gen_fall_feature_sliding(mat, time_step, window_len, nrt_type)):
        feature[i] = item
    return feature

def gen_fall_feature_sliding(mat, time_step, window_len, nrt_type = "instantaneous"):
    """
    Generator.
    Calculate features for fall detection.
    The feature includes;
    1: non response time of all sensors at the present time
    2: maximum height of the edge of the non response time sequence in the left / right part of the window.

    Parameters
    ----------
    mat : nummpy.ndarray of bool
        Raw sensor data matrix.
        mat[i][j] = j-th sensor state at i-th time.
    time_step : float
        Time step length [seconds] of mat = mat[1][0] - mat[0][0].
    window_len : int
        Length of window.
        window_len is the number of columns in mat.
        window_len must be an odd number.
    mode : int
        "instantaneous" : instantaneous non response time
        "sum" : sum in the window
        "max" : max in the window
    
    Yields
    ------
    feature : numpy.ndarray
        (dimension of feature, ).
    """

    def edge_detection(window_data):
        """
        Parameters
        ----------
        window_data : numpy.ndarray
            Non response time. The shape is (window_len, num. of sensors).

        Returns
        -------
        (left_edge_height, right_edge_height) : tuple of int
            left_edge_height is a maximum height of the edge in the left part of the window.
        """
        # Flatten the window by calculating max value among all sensors
        window_data = np.max(window_data, axis = 1)
        # convolution for edge detection
        _filter = np.array([-1, 1])
        edge = np.convolve(window_data, _filter, 'valid')
        half_window_len = int((len(window_data) - 1) / 2)
        left_edge_height = np.max(edge[:half_window_len])
        right_edge_height = np.max(edge[half_window_len:])
        return (left_edge_height, right_edge_height)
    
    def summarize_window(window_data):
        if nrt_type == "sum":
            return np.count_nonzero(window_data, axis=0)
        elif nrt_type == "max":
            return np.max(window_data, axis = 0)
        elif nrt_type == "instantaneous":
            center = int((window_data.shape[0] - 1) / 2)
            return window_data[center]

    # Initialize the output array
    sensor_num = mat.shape[1]
    last_fired_time = np.full((sensor_num,), -1)
    last_fired_sensors = np.zeros(sensor_num, dtype = bool)
    
    w = int((window_len - 1)/2)
    window_data = np.zeros((window_len, sensor_num))

    for i in range(w):
        sd = mat[i]
        if np.any(sd):
            for s in range(sensor_num):
                if not last_fired_sensors[s] and sd[s]:
                    last_fired_time[s] = i
                if last_fired_sensors[s] and not sd[s]:
                    last_fired_time[s] = -1
            last_fired_sensors = sd

        instantaneous_nrt = np.empty(sensor_num)
        for s in range(sensor_num):
            instantaneous_nrt[s] = ((i - last_fired_time[s] + 1) if last_fired_time[s] != -1 else 0)
        window_data[i+w+1, :] = instantaneous_nrt

    # [0, 0, 0, a, b] for example of w = 2.

    for i in range(w, mat.shape[0]):
        utils.print_progress_bar(mat.shape[0] - 1, i, "Extract fall features.", step = 1000)
        # center = i - w  # index of window center
        sd = mat[i]
        if np.any(sd):
            for s in range(sensor_num):
                if not last_fired_sensors[s] and sd[s]:
                    last_fired_time[s] = i
                if last_fired_sensors[s] and not sd[s]:
                    last_fired_time[s] = -1
            last_fired_sensors = sd

        instantaneous_nrt = np.empty(sensor_num)
        for s in range(sensor_num):
            instantaneous_nrt[s] = ((i - last_fired_time[s] + 1) if last_fired_time[s] != -1 else 0)

        # shift window
        # For example of w = 2;
        # from [a, b, c, d, e] to [b, c, d, e, f]
        window_data = np.roll(window_data, shift=-1, axis=0)
        window_data[-1, :] = instantaneous_nrt

        # Calculate elapsed time of the sliding window
        yield summarize_window(window_data) * time_step

    for center in range(mat.shape[0]-w, mat.shape[0]):
         # shift window
        window_data = np.roll(window_data, shift=-1, axis=0)
        window_data[-1, :] = np.zeros(sensor_num)

        # Calculate elapsed time of the sliding window
        yield summarize_window(window_data) * time_step

def extract_data_with_fall_feature_sliding(path, data_type = "raw", time_step = 1, window_len = 121, nrt_type = "instantaneous", data_range = "full", half_len = 10000):
    if data_range not in ["full", "around_anomalies"]:
        raise ValueError("data_range is invalid value!")
    
    SD_model = utils.pickle_load(path, "SD_model")
    # SD = utils.pickle_load(path / "experiment", f"reduced_301_SD_mat_{data_type}_1")
    SD = utils.pickle_load(path / "experiment", f"SD_mat_raw_5")    # !!!!!!!!!!!!!!!!!!!!
    AL = utils.pickle_load(path / "experiment", f"AL_mat_raw_5")
    SD_names = utils.pickle_load(path / "experiment", "SD_names")

    SD = SD[:, [i for i in range(SD.shape[1]) if i != 6]]

    # extract motion sensor data
    motion_sensor_indexes = []
    for i, s_i in enumerate(SD_names):
        if SD_model[s_i].type_name in ['PIR', 'pressure', 'door']:
            motion_sensor_indexes.append(i)
    SD = SD[:, motion_sensor_indexes]

    # extract fall labels
    anomaly_index = 4
    AL = AL[:, anomaly_index]
    
    # extract features of non response time
    if data_range == "full":
        X = fall_feature_sliding(SD, time_step=time_step, window_len=window_len, nrt_type=nrt_type)
        return X, AL, motion_sensor_indexes
    elif data_range == "around_anomalies":
        fall_indices = analysis.find_true_regions_in_ndarray(AL)
        X = np.empty((0, len(motion_sensor_indexes)))
        y = []
        for r in fall_indices:
            start, end = r[0] - half_len, r[1] + half_len
            if (start < 0) or (SD.shape[0] < end):
                continue
            feature = fall_feature_sliding(SD[start:end], time_step=time_step, window_len=window_len, nrt_type=nrt_type)
            X = np.vstack((X, feature))
            for i in range(start, end):
                y.append(True in [r[0] <= i < r[1] for r in fall_indices])
        return X, np.array(y), motion_sensor_indexes
    
    
def online_detection_test(classifier, path, data_type = "raw", time_step = 1, window_len = 121, nrt_type = "instantaneous", data_range = "full", half_len = 10000):
    """
    
    Returns
    -------
    (y_true, y_pred) : tuple of numpy.ndarray
    """
    
    
    if (data_range not in ["full", "around_anomalies"]) and not isinstance(data_range, tuple):
        raise ValueError("data_range is invalid value!")
    
    SD_model = utils.pickle_load(path, "SD_model")
    # SD = utils.pickle_load(path / "experiment", f"reduced_301_SD_mat_{data_type}_{time_step}")
    SD = utils.pickle_load(path / "experiment", f"SD_mat_{data_type}_{time_step}")
    AL = utils.pickle_load(path / "experiment", f"AL_mat_{data_type}_{time_step}")
    SD_names = utils.pickle_load(path / "experiment", "SD_names")

    SD = SD[:, [i for i in range(SD.shape[1]) if i != 6]]

    # extract motion sensor data
    motion_sensor_indexes = []
    for i, s_i in enumerate(SD_names):
        if SD_model[s_i].type_name in ['PIR', 'pressure', 'door']:
            motion_sensor_indexes.append(i)
    SD = SD[:, motion_sensor_indexes]

    # extract fall labels
    anomaly_index = 4
    AL = AL[:, anomaly_index]

    # extract features of non response time
    if data_range == "full":
        y_pred = np.zeros(SD.shape[0], dtype = bool)
        for i, item in enumerate(gen_fall_feature_sliding(SD, time_step, window_len, nrt_type)):
            y_pred[i] = classifier.predict(item.reshape(1, -1))
        return AL, y_pred
    elif data_range == "around_anomalies":
        fall_indices = analysis.find_true_regions_in_ndarray(AL)
        y_true = []
        y_pred = []
        for r in fall_indices:
            start, end = r[0] - half_len, r[1] + half_len
            if (start < 0) or (SD.shape[0] < end):
                continue
            for i, item in enumerate(gen_fall_feature_sliding(SD[start:end], time_step, window_len, nrt_type)):
                y_pred.append(classifier.predict(item.reshape(1, -1)))
            for i in range(start, end):
                y_true.append(True in [r[0] <= i < r[1] for r in fall_indices])
        return y_true, y_pred
    elif isinstance(data_range, tuple):
        start, end = data_range[0], data_range[1]
        y_pred = np.zeros(end-start, dtype = bool)
        for i, item in enumerate(gen_fall_feature_sliding(SD[start:end], time_step, window_len, nrt_type)):
            y_pred[i] = classifier.predict(item.reshape(1, -1))
        return AL[start:end], y_pred


def online_detection_rule(path, data_type = "raw", time_step = 1, window_len = 121, nrt_type = "instantaneous", data_range = "full", half_len = 10000):
    """
    
    Returns
    -------
    (y_true, y_pred) : tuple of numpy.ndarray
    """


    def classifier_rule(vec, valid_range):
        for i, v in enumerate(vec):
            if (i in valid_range) and v >= 20:
                return True
        else:
            False    
    
    if (data_range not in ["full", "around_anomalies"]) and not isinstance(data_range, tuple):
        raise ValueError("data_range is invalid value!")
    
    SD_model = utils.pickle_load(path, "SD_model")
    # SD = utils.pickle_load(path / "experiment", f"reduced_301_SD_mat_{data_type}_{time_step}")
    SD = utils.pickle_load(path / "experiment", f"SD_mat_{data_type}_{time_step}")
    AL = utils.pickle_load(path / "experiment", f"AL_mat_{data_type}_{time_step}")
    SD_names = utils.pickle_load(path / "experiment", "SD_names")

    # extract motion sensor data
    motion_sensor_indexes = []
    for i, s_i in enumerate(SD_names):
        if SD_model[s_i].type_name in ['PIR', 'pressure', 'door']:
            motion_sensor_indexes.append(i)
    SD = SD[:, motion_sensor_indexes]

    # extract fall labels
    anomaly_index = 4
    AL = AL[:, anomaly_index]

    valid_range = set(range(0, 22)) + {33, 34} 

    # extract features of non response time
    if data_range == "full":
        y_pred = np.zeros(SD.shape[0], dtype = bool)
        for i, item in enumerate(gen_fall_feature_sliding(SD, time_step, window_len, nrt_type)):
            y_pred[i] = classifier_rule(item, valid_range)
        return AL, y_pred
    elif data_range == "around_anomalies":
        fall_indices = analysis.find_true_regions_in_ndarray(AL)
        y_true = []
        y_pred = []
        for r in fall_indices:
            start, end = r[0] - half_len, r[1] + half_len
            if (start < 0) or (SD.shape[0] < end):
                continue
            for i, item in enumerate(gen_fall_feature_sliding(SD[start:end], time_step, window_len, nrt_type)):
                y_pred.append(classifier_rule(item, valid_range))
            for i in range(start, end):
                y_true.append(True in [r[0] <= i < r[1] for r in fall_indices])
        return y_true, y_pred
    elif isinstance(data_range, tuple):
        start, end = data_range[0], data_range[1]
        y_pred = np.zeros(end-start, dtype = bool)
        for i, item in enumerate(gen_fall_feature_sliding(SD[start:end], time_step, window_len, nrt_type)):
            y_pred[i] = classifier_rule(item, valid_range)
        return AL[start:end], y_pred

data_type = "raw"
time_step = 1
window_len = 121
# half_len = 5000
half_len = 30000
nrt_type = "instantaneous"

# training data
# train_path = layout_database_path / "test_layout" / "test_data_3"
# X_train, y_train, motion_sensor_indexes  = extract_data_with_fall_feature_sliding(train_path, 
#         data_type = data_type, time_step = time_step, window_len = window_len, nrt_type = nrt_type, data_range = "around_anomalies", half_len = half_len)

train_path = layout_database_path / "test_layout" / "fall_test_2"
X_train, y_train, motion_sensor_indexes  = extract_data_with_fall_feature_sliding(train_path, 
        data_type = data_type, time_step = time_step, window_len = window_len, nrt_type = nrt_type, data_range = "full", half_len = half_len)
print("Training data is ready!")

# decision tree
decision_tree = DecisionTreeClassifier(min_samples_leaf=5)
decision_tree.fit(X_train, y_train)
print("Classifier is ready!")

# test data
test_path = layout_database_path / "test_layout" / "test_data_5"

# classfy after generating test data
# X_test, y_true, motion_sensor_indexes = extract_data_with_fall_feature_sliding(test_path,
#         data_type = data_type, time_step=time_step, window_len=window_len, nrt_type = nrt_type, data_range = "around_anomalies", half_len = half_len)
# print("Test data is ready!")
# y_test_pred = decision_tree.predict(X_test)

# y_test, y_pred = online_detection_test(decision_tree, test_path,
#         data_type = data_type, time_step=time_step, window_len=window_len, nrt_type = nrt_type, data_range = "around_anomalies", half_len = half_len)

# y_pred = decision_tree.predict(X_train)
# print("Training error ------------------------")
# print("Confusion matrix:")
# print(confusion_matrix(y_train, y_pred))
# print("Classification report")
# print(classification_report(y_train, y_pred))

# print("Test error ------------------------")
# print("Confusion matrix:")
# print(confusion_matrix(y_test, y_pred))
# print("Classification report")
# print(classification_report(y_test, y_pred))

In [ ]:
end = 9*360*24*60*60
start = end - 3*30*24*60*60
test_path = layout_database_path / "test_layout" / "test_data_5"
y_test_r, y_pred_r = online_detection_rule(test_path,
        data_type = data_type, time_step=time_step, window_len=window_len, nrt_type = nrt_type, data_range = (start, end), half_len = half_len)

y_test_d, y_pred_d = online_detection_test(decision_tree, test_path,
        data_type = data_type, time_step=time_step, window_len=window_len, nrt_type = nrt_type, data_range = (start, end), half_len = half_len)

# y_pred = decision_tree.predict(X_train)
# print("Training error ------------------------")
# print("Confusion matrix:")
# print(confusion_matrix(y_train, y_pred))
# print("Classification report")
# print(classification_report(y_train, y_pred))

print("Test error Rule------------------------")
print("Confusion matrix:")
print(confusion_matrix(y_test_r, y_pred_r))
print("Classification report")
print(classification_report(y_test_r, y_pred_r))


print("Test error DT------------------------")
print("Confusion matrix:")
print(confusion_matrix(y_test_d, y_pred_d))
print("Classification report")
print(classification_report(y_test_d, y_pred_d))